In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tensorflow.keras.utils import to_categorical
from keras.layers import Dropout, Dense
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, Callback, LearningRateScheduler


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# overview 
train_data=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_data=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
test_id=test_data['PassengerId']

train_data.drop(labels='Name',axis=1, inplace=True)
#train_data.drop(labels='PassengerId',axis=1, inplace=True)

test_data.drop(labels='Name',axis=1, inplace=True)
#test_data.drop(labels='PassengerId',axis=1, inplace=True)

train_data.head()

要舍去的数据列为id，名字，也可以不舍去，没时间分析了就舍去吧

In [ ]:
# show null presentage
# train_data.dropna(axis=0, how='any', inplace=True)
print((train_data.isnull().sum()).sort_values(ascending=False))
print(train_data.shape)
print(test_data.info())

缺失的数据极少，可以用平均值,众数补全

In [ ]:
# fix missing data
# numeric use mean

def fix_missing(data):
    numeric_cols = [column for column in data.select_dtypes(["int", "float"])]
    for j in numeric_cols:
        data[j].fillna(data[j].mean(), inplace=True)
    categoric_cols = [column for column in data.select_dtypes(exclude = ["int", "float"])]
    for j in categoric_cols:
        data[j].fillna(data[j].value_counts().index[0], inplace=True)
    return data
        
train_data_nonull = fix_missing(train_data)
test_data_nonull = fix_missing(test_data)
#print((train_data.isnull().sum()).sort_values(ascending=False))
train_data_nonull.head()

处理数据，转化为int类型，某些特征因子化变成onehot形式，有些值太多暂时保留

In [ ]:

# processing 
def processing_data(data):
    destination = pd.get_dummies(data['Destination'],prefix='Des')
    data = pd.concat([data, destination], axis=1)
    
    home = pd.get_dummies(data['HomePlanet'], prefix='Home')
    data = pd.concat([data, home], axis=1)
    
    data.drop(labels='Destination', axis=1, inplace=True)
    data.drop(labels='HomePlanet', axis=1, inplace=True)
    
    data['group_num']=data['PassengerId'].apply(lambda x:x.split('_')[0]).astype('int')
    data['in_group_num']=data['PassengerId'].apply(lambda x:x.split('_')[1]).astype('int')
    
    data['deck']=data['Cabin'].apply(lambda x:x.split('/')[0])
    data['num']=data['Cabin'].apply(lambda x:x.split('/')[1])
    data['side']=data['Cabin'].apply(lambda x:x.split('/')[2])
    
    data['num']=data['num'].astype('int')
    
    data['CryoSleep']=data['CryoSleep'].map({False:0, True:1})
    data['VIP']=data['VIP'].map({False:0, True:1})
    data['side']=data['side'].map({'P':0, 'S':1})
    
    data.drop(labels='Cabin', axis=1, inplace=True)
    data.drop(labels='PassengerId', axis=1, inplace=True)
    
    deck = pd.get_dummies(data['deck'], prefix='deck')
    data = pd.concat([data, deck], axis=1)
    data.drop(labels='deck', axis=1, inplace=True)
    
    
    return data

data_new_train = processing_data(train_data_nonull)
data_new_test = processing_data(test_data_nonull)

data_new_train.head()


正则化

In [ ]:
def scaler_data(train_data,test_data):
    target_data = train_data['Transported']
    feature_data = train_data.copy()
    feature_data_t = test_data.copy()
    feature_data.drop(labels='Transported', axis=1, inplace=True)
    all_data = pd.concat([feature_data, feature_data_t])
    
    float_col=[col for col in all_data.select_dtypes(["float"])]
    scaler = StandardScaler()
    for j in float_col:
        scaler_params = scaler.fit(all_data[j].values.reshape(-1, 1))
        feature_data[j] = scaler.transform(feature_data[j].values.reshape(-1, 1), scaler_params)
        feature_data_t[j] = scaler.transform(feature_data_t[j].values.reshape(-1, 1), scaler_params)
    return feature_data,target_data,feature_data_t
    
    
scaled_train_feature, scaled_train_target, scaled_test_feature = scaler_data(data_new_train,data_new_test)
scaled_train_feature.head()

xgboost try

In [ ]:
def xgboost_train_try(x,y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                        test_size=0.1, 
                                                        shuffle=True, 
                                                        random_state = 1)
    model = xgb.XGBClassifier(gamma = 1.5,
                              subsample = 1.0,
                              max_depth = 5,
                              colsample_bytree = 1.0,
                              n_estimators = 200,
                              silent=1,
                              objective='binary:logistic'
                             )
    model.fit(x_train,y_train,early_stopping_rounds=10, eval_metric='logloss', eval_set=[(x_test, y_test)],
              verbose=True)
    y_pred = model.predict(x_test)
    acc = accuracy_score(y_test, y_pred)
    xgb.plot_importance(model)
    plt.show()
    print(acc)
    return model
try_model=xgboost_train_try(scaled_train_feature, scaled_train_target)


In [ ]:
def fully_xgboost(x,y):
    model = xgb.XGBClassifier(gamma = 1.5,
                              subsample = 1.0,
                              max_depth = 5,
                              colsample_bytree = 1.0,
                              n_estimators = 50,
                              objective='binary:logistic'
                             )
    model.fit(x,y)
    return model

xgbmodel=fully_xgboost(scaled_train_feature, scaled_train_target)

In [ ]:
# predict with xgboost
prediction=xgbmodel.predict(scaled_test_feature)
pred = pd.Series(prediction).map({0:False, 1:True})
submission=pd.DataFrame({"PassengerId": test_id.values, "Transported": pred})
submission.to_csv('submission_xgboost.csv', index = False)